In [35]:
from experta import *
import random
import re
import copy

In [2]:
enfermedades = {
    "Resfriado común": ["Congestión nasal", "Goteo nasal", "Estornudos", "Dolor de garganta", "Tos", "Fatiga leve"],
    "Gripe (Influenza)": ["Fiebre alta", "Dolor de cabeza", "Dolores musculares y corporales", "Fatiga intensa", "Congestión nasal", "Tos seca"],
    "Fiebre del dengue": ["Fiebre alta", "Dolor de cabeza severo", "Dolor detrás de los ojos", "Erupción cutánea", "Dolor articular y muscular", "Fatiga"],
    "Diabetes tipo 2": ["Aumento de la sed", "Micción frecuente", "Fatiga", "Visión borrosa", "Heridas que sanan lentamente", "Pérdida de peso inexplicada"],
    "Hipertensión arterial": ["Dolor de cabeza", "Visión borrosa", "Fatiga", "Mareos", "Sangrado nasal", "Zumbido en los oídos"],
    "Enfermedad cardíaca coronaria": ["Dolor en el pecho (angina)", "Dificultad para respirar", "Fatiga", "Mareos", "Sudoración excesiva", "Náuseas"],
    "Asma": ["Sibilancias", "Tos", "Dificultad para respirar", "Opresión en el pecho", "Respiración rápida y superficial", "Tos nocturna"],
    "Enfermedad de Alzheimer": ["Pérdida de memoria a corto plazo", "Confusión", "Dificultad para recordar palabras", "Cambios en el comportamiento", "Dificultad para realizar tareas cotidianas", "Desorientación en tiempo y lugar"],
    "Artritis reumatoide": ["Dolor e hinchazón en las articulaciones", "Rigidez matutina", "Fatiga", "Deformidades en las articulaciones", "Pérdida de apetito", "Fiebre leve"],
    "Enfermedad de Crohn": ["Dolor abdominal", "Diarrea crónica", "Fatiga", "Pérdida de peso", "Fiebre", "Úlceras en el tracto digestivo"]
}

In [8]:
all_symptoms = []
for s_list in enfermedades.values():
    all_symptoms = all_symptoms + s_list
all_symptoms = list(set(all_symptoms))

In [56]:
def do_illness_choice(symptom):
    have_symptom = []
    for k, v in enfermedades.items():
        for s in v:
            if re.search(symptom, s, re.IGNORECASE):
                have_symptom.append(k)
    return random.choice(have_symptom), have_symptom

In [72]:
def get_question(symptom):
    return f"¿Tiene {symptom}? (s/n)"

class IllnessRobot(KnowledgeEngine):
    @DefFacts()
    def _initial_action(self):
        yield Fact(action="greet")
        
    @Rule(Fact(action='greet'),
          NOT(Fact(name=W())))
    def do_welcome(self):
        print("Bienvenido@ al chat de enfermedades, a continuación se presentará la primera pregunta")
        self.declare(Fact(introduced=True))
        
    @Rule(OR(Fact(introduced=True),Fact(answer="n", symptom=W(), salt=W()), Fact(redo=W())))
    def first_question(self):
        symptom = random.choice(all_symptoms)
        self.declare(Fact(answer=input(get_question(symptom)), symptom=symptom, salt=random.gauss(0,1)))
        
    @Rule(Fact(answer="s", symptom=MATCH.symptom))
    def do_first_illness_guess(self, symptom):
        illness, illnesses = do_illness_choice(symptom)
        symptoms = copy.deepcopy(enfermedades[illness])
        symptoms.remove(symptom)
        strikes = 0
        self.declare(Fact(illness=illness, illnesses=illnesses, symptoms=symptoms, strikes=strikes, answer="s"))

    @Rule(Fact(illness=MATCH.illness, illnesses=MATCH.illnesses, symptoms=MATCH.symptoms, strikes=MATCH.strikes, answer=MATCH.answer))
    def questions_engine(self, illness, illnesses, symptoms, strikes, answer):
        strikes += 1 if answer == "s" else -1
        if strikes == -4:
            self.declare(Fact(redo=random.gauss(0,1)))
        elif strikes == 4:
            print(f"Usted podría tener {illness}.")
        else:
            if len(symptoms) == 0:
                illnesses = list(illnesses)
                illnesses.remove(illness)
                if len(illnesses) > 0:
                    print(f"Usted podría tener síntomas para las enfermedades {', '.join(illnesses)}")
                else:
                    print(f"No se logró encontrar una enfermedad con los síntomas indicados")
            else:
                symptoms = list(symptoms)
                s_to_ask = random.choice(symptoms)
                symptoms.remove(s_to_ask)
                self.declare(Fact(illness=illness,illnesses=illnesses, symptoms=symptoms, strikes=strikes, answer=input(get_question(s_to_ask))))

In [76]:
engine = IllnessRobot()
engine.reset()
engine.run()

Bienvenido@ al chat de enfermedades, a continuación se presentará la primera pregunta


¿Tiene Tos seca? (s/n) n
¿Tiene Tos nocturna? (s/n) n
¿Tiene Confusión? (s/n) n
¿Tiene Dolor articular y muscular? (s/n) n
¿Tiene Deformidades en las articulaciones? (s/n) n
¿Tiene Cambios en el comportamiento? (s/n) n
¿Tiene Congestión nasal? (s/n) s
¿Tiene Dolor de garganta? (s/n) n
¿Tiene Tos? (s/n) n
¿Tiene Goteo nasal? (s/n) n
¿Tiene Fatiga leve? (s/n) n
¿Tiene Estornudos? (s/n) n
¿Tiene Respiración rápida y superficial? (s/n) n
¿Tiene Pérdida de peso? (s/n) n
¿Tiene Dolor de garganta? (s/n) n
¿Tiene Dificultad para recordar palabras? (s/n) n
¿Tiene Sibilancias? (s/n) n
¿Tiene Fiebre leve? (s/n) n
¿Tiene Dolores musculares y corporales? (s/n) n
¿Tiene Pérdida de peso inexplicada? (s/n) n
¿Tiene Dolor de garganta? (s/n) n
¿Tiene Pérdida de peso inexplicada? (s/n) n
¿Tiene Estornudos? (s/n) n
¿Tiene Pérdida de memoria a corto plazo? (s/n) n
¿Tiene Zumbido en los oídos? (s/n) n
¿Tiene Fiebre? (s/n) s
¿Tiene Diarrea crónica? (s/n) n
¿Tiene Úlceras en el tracto digestivo? (s/n) s
¿Tien

Usted podría tener síntomas para las enfermedades Gripe (Influenza), Fiebre del dengue, Artritis reumatoide
